# Basic manipulation of tabular data structures

Two of the simplest operations over tabular data are to choose only certain columns from the tables, or to choose certain rows - we're going to look at these operations, known as projection and selection, in the following examples.

We will use *pandas* DataFrames as our table representations in this Notebook and show how the basic projection and selection operations are coded in Python, and in SQL.

In [ ]:
import pandas as pd

The SQL we will use is the _pandas_ SQL which allows us to apply an SQL command to a _pandas_ DataFrame.  We will say more about this when we introduce pandaSQL below.

*Note: one limitation of the pandaSQL, because of its underlying use of sqlite, is that it cannot handle DataFrame column names that contain spaces.   So our examples have avoided this problem by avoiding column names with spaces in them.*

## The example DataFrames used in this Notebook 
In the following examples we'll be using one of the following two tables created as DataFrames (CourseData, and ABCD) to show the code and the behaviour of the code.

In [ ]:
# Create the courseData DataFrame:
courseData = { 'courseCode':['TM351','TU100','M269'],
              'points':[30,60,30],
              'level':['3','2','1']
             }
course_df = pd.DataFrame(courseData)
course_df

In [ ]:
# Create the ABCD DataFrame:
ABCD = {'A':['a1','a2','a9'],
        'B':['b1','b4','b5'],
        'C':['c1','c7','c7'],
        'D':['c1','d9','d7']
       }
ABCD_df = pd.DataFrame(ABCD)
ABCD_df            

Remember that the index column is added automatically to *pandas* DataFrames.


## Setting up pandasql

In [ ]:
# Start by importing the sqldf function from pandasql:
from pandasql import sqldf

# To make it a bit easier to apply the sqldf function, we will create a 
# simple wrapper function to allow us to supply the query 'q' without the 
# surrounding syntax of the function call.
pysqldf = lambda q: sqldf(q, globals())

`pysqldf` is created as a simple wrapper function that takes a query over one or more *pandas* DataFrames, pops the DataFrames into an sqlite3 database (a lightweight, file-based relational database), runs the query over the DataFrames-recast-as-sqlite-tables, and returns the result as a *pandas* DataFrame.

Note: we don't generally recommend using pandasql when working directly with *pandas* DataFrames - use the more direct *pandas* constructions directly.  However, if you are having difficulty with the *pandas* syntax or thinking through how to construct a complex operation, you may find it easier to write the equivalent SQL expression down first as pseudocode, and then translate that into the *pandas* equivalent one step at a time. 

To create and apply an SQL query we create a string variable containing the SQL command, and use the wrapper function we just created, `pysqldf`, to evaluate it.

In [ ]:
query = '''SELECT * FROM course_df;'''
pysqldf(query)

SQL uses the SELECT statement for most operations that manipulate tables - over the course of the next couple of weeks the SELECT statement will gradually be explored in more detail.  In the above example we are using it in its most basic form - to extract all the data in all the rows and columns of a table.

## PROJECTION: choosing certain columns 

If we want to reshape a table by removing some columns (or conversely only keeping some columns) we will want the rows preserved - this operation is known as projection over a table.
So in our ABCD table we might choose to PROJECT ABCD OVER columns A and C ('over' here simply meaning the resulting table will have those two columns).

### Projection using *pandas*

In *pandas* we simply apply to the DataFrame a list of the column names we want.

In [ ]:
result_df = ABCD_df[['A', 'C']]
result_df

Of course we can change the list order to change the order of the columns in the result.

In [ ]:
result_df = ABCD_df[['C','A']]
result_df

### Activity
Using the `course_df` DataFrame, project `course_df` over the `courseCode` and `level` columns.

In [ ]:
# Your solution here:


In [ ]:
# Solution:
result = course_df[['courseCode', 'level']]
result

### Projection using SQL

The SQL syntax  for projection is different but has the same aim: to give a list of the column names we want to retain in our result.  

In [ ]:
query = ''' SELECT A, C FROM ABCD_df; '''
result_df = pysqldf(query)
result_df

This statement has two parts: the SELECT and the FROM. 

The FROM tells us which table (dataset) is to be used, and the column names after the SELECT tell us the names of the columns from which to keep (project) the values from each row in the table.  

So, in English the statement simply means:

For each row in table ABCD put the A and C values from that row into the result table.

SQL is (mainly) a declarative programming language, that is, a statement describes the data that comes from its execution, rather than giving a series of instructions saying how to calculate the result. 

Many users familiar with SQL don't realise that a good way to think about SQL is that it processes tables a ‘row-at-a-time’.  When describing what SQL code will do we will use ‘long’ descriptions full  of sentences that begin 'for each row in…'. We refer to this as the logical processing model, or logical execution model. 

However, the SQL execution engine will choose whatever execution sequence it estimates is most efficient, provided it achieves the required result.

SQL also allows columns to be renamed as part of the project operation, as follows.

In [ ]:
query = '''SELECT A, B AS Bcolumn, C AS othercolumn FROM ABCD_df;'''
result_df = pysqldf(query)
result_df

### Activity 
Using the `SQL SELECT` statement, project `course_df` over the `courseCode` and `points` columns and rename the `courseCode` column as `coursecode`.

In [ ]:
# Your solution here.


In [ ]:
# Solution:
query = '''SELECT courseCode AS coursecode, points FROM course_df;'''
result_df = pysqldf(query)
result_df

So, projection is pretty straightward: it's taking vertical slices of the table by listing the names of the columns we want to retain.

## SELECTION: choosing certain rows

Another way to simply reshape a table is to select only certain rows of the original table. This is a form of filtering: we apply a *filter* to each row of the table: if the row passes the filter it appears in the result, otherwise it doesn't. 

To do this we apply a condition to each row in the table: if the condition is true for that row then we want the row in the result table, if the condition is false for that row then we don't want it to appear in the result.

So, in selection the table is sliced horizontally, retaining only the rows we choose.

## Selection using *pandas*

In *pandas* we filter rows by adding a conditional row selection statement to the DataFrame index selection.

In [ ]:
result_df = course_df[course_df['points']==30]
result_df

And of course, since the result is a DataFrame we can project the result over chosen columns at the same time.

In [ ]:
result_df = course_df[course_df['points']==30][['courseCode','level']]
result_df

### Activity

In the `ABCD_df` DataFrame select those rows where column C has the value 'c7'.

In [ ]:
# Your solution here:


In [ ]:
# Solution:
result_df = ABCD_df[(ABCD_df['C']=='c7')]
result_df

We can also apply multiple conditions. Here's the code to select rows that have `b1` or `b4` in the `B` column

In [ ]:
result_df = ABCD_df[(ABCD_df['B']=='b1') | (ABCD_df['B']=='b4')]
result_df

## Selection using SQL

In SQL the row condition is placed in a WHERE clause following the FROM. (*Note*: we've used whitespace to lay out the SQL query over several lines.  This is a common layout, with each clause of the SQL on a new line, as it helps readablity.)

In [ ]:
query= ''' SELECT A,B,C,D 
           FROM ABCD_df 
           WHERE C='c7'; 
       '''
result_df = pysqldf(query)
result_df

If typing all the column names from the original table looks time consuming, SQL has the shorthand of * in the SELECT clause standing for 'project over all columns'.

In [ ]:
query= ''' SELECT * 
           FROM ABCD_df 
           WHERE C='c7'; 
       '''
result_df = pysqldf(query)
result_df

The condition in the WHERE clause can be as complex as we need it to be.  So we can ask for the courses that are 30 credits and level 1.

In [ ]:
query= ''' SELECT * 
           FROM course_df 
           WHERE points = 30 AND level = '1'; 
       '''
result_df = pysqldf(query)
result_df

And here's SQL using the OR condition.

In [ ]:
query= ''' SELECT * 
           FROM ABCD_df 
           WHERE B = 'b1' OR B = 'b4'; 
       '''
result_df = pysqldf(query)
result_df

### Activity
Write (a) SQL and (b) Python to select only those rows of `course_df` that do not contain a 30-credit course.

In [ ]:
# Your solution (a)


In [ ]:
# Your solution (b)


In [ ]:
# Solution (a):
query = '''SELECT *
           FROM course_df
           WHERE NOT (points = 30);
        '''
result_df = pysqldf(query)
result_df

In [ ]:
# Altenative solution (a):
query = '''SELECT *
           FROM course_df
           WHERE points <> 30;
        '''
result_df = pysqldf(query)
result_df

In [ ]:
# Solution (b):
result_df = course_df[course_df['points']!=30]
result_df

# Limiting the number of rows displayed

Both *pandas* and SQL allow you to specify a limit to the number of rows in the result.  This is a fairly abritrary operation however, as SQL doesn't make any guarantees what order the rows are stored in, so unless explicitly sorting the table rows first you won't know which rows you will get.  However, it can be useful when debugging complex queries that result in large tables. It lets you see a sample of what the query is doing, without having to see the whole table.

In *pandas* the `head()` function applied to a DataFrame shows the rows from the beginning of the DataFrame:

In [ ]:
course_df.head(2)

And there is also the [M:N] notation which lets you choose which rows to display:

In [ ]:
print(course_df[0:2])
print(course_df[:4])
print(course_df[2:])

In SQL the LIMIT clause is added to end of the query. Remember, you cannot be sure which rows are displayed as SQL shouldn't have any sense of internal order.

In [ ]:
query = ''' SELECT *
            FROM course_df
            LIMIT 2;    
        '''
result_df = pysqldf(query)
result_df


# Sorting the rows displayed

Again, both *pandas* and SQL allow the table rows to be sorted based on the values in specified columns (and in both cases you can choose ascending or descending sort order).

In [ ]:
result_df = course_df.sort_values(by=['courseCode'])
result_df

In [ ]:
result_df = course_df.sort_values(by=['courseCode'], ascending=False)
result_df

In [ ]:
# Multi-column sorts (with individual directions possible for each column):
result_df = course_df.sort_values(by=['points', 'level'], ascending=[True, False])
result_df

And now the SQL equivalents of these three examples.

In [ ]:
query = ''' SELECT *
            FROM course_df
            ORDER BY courseCode;
        '''
result_df = pysqldf(query)
result_df

In [ ]:
query = ''' SELECT *
            FROM course_df
            ORDER BY courseCode DESC;    
        '''
result_df = pysqldf(query)
result

In [ ]:
query = ''' SELECT *
            FROM course_df
            ORDER BY points ASC, courseCode DESC;    
        '''
result_df = pysqldf(query)
result_df

You may have noticed that the index columns in the *pandas* results and the SQL results are different.  The *pandas* `sort()` method is being applied directly to the DataFrame object, so sorting retains the original row indexes from the `course_df` DataFrame.  In pandasql the original DataFrame is first converted into an sqlite table, over which the query is evaluated. The result is then put into a new DataFrame, so the index is new for the pandasql result.

# What next?

In this Notebook, you have seen examples of several techniques for selecting certain columns or certain rows from a table structure.

This is the basis of manipulating tables based on the attributes of the tabular data, or the specific values of the data elements.  Feel free to add to this Notebook as you come up with your own table-manipulation recipes.

If you are working through this Notebook as part of an inline exercise, return to the module materials now. If you are working through this set of Notebooks as a whole, move on to `03.3 Combining data from multiple datasets`.